*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Model Ensembling

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/model_ensembling.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This notebook provides an example of how to ensemble multiple recommendation models in Cornac.

Ensemble models is a technique that combines the predictions of multiple models to produce a single prediction. The idea is that by combining the predictions of multiple models, we can improve the overall performance of the recommendation system.

We will use the MovieLens 100K dataset and ensemble 2 models.

** Note: ** Part of this notebook (in Section 4) uses the `scikit-learn` package. 

## 1. Setup

### 1.1 Install required dependencies

In [23]:
! pip install seaborn scikit-learn

In [2]:
from IPython.display import display
import numpy as np
import pandas as pd

from cornac.datasets import movielens
from cornac.models import BPR, WMF
from cornac.eval_methods import RatioSplit
from cornac.metrics import Precision, Recall
from cornac.utils import cache
from cornac import Experiment

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

/Users/darrylong/miniconda3/envs/gcmc-pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Prepare Experiment

### 2.1 Loading Dataset

First, we load the MovieLens 100K dataset.

In [3]:
data = movielens.load_feedback(variant="100K") # Load MovieLens Dataset

rs = RatioSplit(data, test_size=0.2, seed=42, verbose=True) # Split to train-test set to 80-20
train_set, test_set = rs.train_set, rs.test_set

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


### 2.2 Training BPR and WMF models

We will train two models: 

1. BPR (Bayesian Personalized Ranking)
2. WMF (Weighted Matrix Factorization)

In [4]:
bpr_model = BPR(k=10, max_iter=100, learning_rate=0.01, lambda_reg=0.001) # Initialize BPR model
wmf_model = WMF(k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01,) # Initialize WMF model

models = [bpr_model, wmf_model]
metrics = [Precision(k=50), Recall(k=50)] # Set metrics for experiment

experiment = Experiment(rs, models, metrics, user_based=True).run() # Run Experiment to compare BPR model to WMF model individually


[BPR] Training started!

[BPR] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1397.81it/s]



[WMF] Training started!


100%|██████████| 300/300 [00:17<00:00, 16.78it/s, loss=174]


Learning completed!

[WMF] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1244.13it/s]



TEST:
...
    | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------------ + --------- + --------- + --------
BPR |       0.1819 |    0.5072 |    1.6507 |   0.6812
WMF |       0.1734 |    0.5079 |  105.8213 |   0.7683



Comparing Precision and Recall, both BPR and WMF are providing comparable results.

Let's move on to try to interpret these results by using the genres of movies that were recommended to us.

Generally, we could assume that if an individual likes a particular film genre like 'Romance', the recommender system should provide more of such 'Romance' films.

### 2.3 Interpreting Results

##### Creating a Movie Genre Dataframe

In [5]:
# Creating a dataframe of movies with its corresponding genres

# Download some information of MovieLens 100K dataset
item_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"), 
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL", 
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", 
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", 
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])

item_idx2id = list(train_set.item_ids) # create a item index to film ID mapping

# Let's take a look at an example of this dataframe
display(item_df.head(3))


,Title,Release Date,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
ItemID,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


The `item_df` dataframe consists of all movie items with its corresponding genre attributes.

Further down below, we are going to filter this table with the recommendations that we get from the recommender system models we created to get a better sense of its performance.

##### Creating Training Data Dataframe

To get a sense of what data has been inserted into our model for training, let's count the genres of the training data used to train the model.

But first, let's create a `training_data_df` dataframe with all training data.

The training data consists of 80000 triplets of **User Index**, **Item Index** and **Rating** rows as seen in the dataset summary in Section 2.1.

In [14]:
# Let's view a sample of the training data dataframe
print("Sample row of record:")
print("(user_index, item_index, rating):", list(zip(*train_set.uir_tuple))[0])

# Create a training data dataframe
training_data_df = pd.DataFrame(zip(*train_set.uir_tuple)) # adding all training data into dataframe
training_data_df.columns = ['user_idx', 'item_idx', 'rating'] # adding column names to the data

# Add new column, 'item_id', for further filtering in later sections
training_data_df['item_id'] = training_data_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1) # converted from the item index field

Sample row of record:
(user_index, item_index, rating): (0, 0, 4.0)


##### Filtering Training Data

Let's filter based on a particular user to learn more about the user.

We set ``UIDX`` to user index **3**, and ``TOPK`` to **50**, to get the top 50 recommendations in each model for comparison.

In [15]:
UIDX = 3
TOPK = 50

In [16]:
# Filter training data (rating = 5.0 and user index = UIDX)
filter_df = training_data_df[(training_data_df['rating'] == 5.0) & (training_data_df['user_idx'] == UIDX)]
filter_df = item_df.loc[[int(item_id) for item_id in filter_df["item_id"]]] # get genres of movie items

print("Number of movies:", len(filter_df))

# Group by Movie Genre and Sum by genres
filter_df = filter_df.select_dtypes(np.number).sum() 
filter_df = filter_df.to_frame("Sum") # Let's call that column 'Sum'

# Add a new column '%' for the percentage of individual genre sum compared to total sum
filter_df["%"] = filter_df["Sum"] / filter_df["Sum"].sum() * 100
filter_df["%"] = filter_df["%"].round(1)

# Let's see the training data genres, sums and percentages
print("Movies rated 5.0 by user index 3 in training data")
display(filter_df.sort_values("Sum", ascending=False))

Number of movies: 138
Movies rated 5.0 by user index 3 in training data


,Sum,%
Drama,71,24.9
Comedy,39,13.7
Romance,32,11.2
Action,30,10.5
Thriller,29,10.2
Adventure,19,6.7
War,15,5.3
Crime,12,4.2
Sci-Fi,9,3.2
Mystery,8,2.8


As shown above in the training data, the top genres for user index 3 with movies rated 5.0 include 'Drama', 'Comedy', 'Romance', 'Action' and 'Thriller'.

Let's now compare them to the recommendations of the BPR and WMF models respectively.

##### Interpreting Recommendations of BPR, WMF Models

In [18]:
# Get the Top 5 Genres in filtered training data for user index 3
top_genres = filter_df.sort_values("Sum", ascending=False).head(5).index.tolist()
print("\nTop 5 Genres in training data:", top_genres)

# Get top K recommendations for BPR and put them into the genre dataframe
bpr_recommendations, bpr_scores = bpr_model.rank(UIDX, k=TOPK) # rank recommendations by score, limit to top K
print(len(bpr_recommendations))
bpr_topk = [item_idx2id[iidx] for iidx in bpr_recommendations[:TOPK]] # convert item indexes into item ids
bpr_df = item_df.loc[[int(iid) for iid in bpr_topk]] # filter the movie genre dataframe by item ids

# Let's view the top recommendations for BPR by top genres
display("BPR: Top recommendations", bpr_df[["Title"] + top_genres].head(10))

# Now, let's do likewise for WMF - get top K recommendations and put them into the genre dataframe
wmf_recommendations, wmf_scores = wmf_model.rank(UIDX, k=TOPK) # rank recommendations by score
wmf_topk = [item_idx2id[iidx] for iidx in wmf_recommendations[:TOPK]] # convert item indexes into item ids
wmf_df = item_df.loc[[int(iid) for iid in wmf_topk]] # filter the movie genre dataframe by item ids

# View the top recommendations for WMF
display("WMF: Top recommendations", wmf_df[["Title"] + top_genres].head(10))


Top 5 Genres in training data: ['Drama', 'Comedy', 'Romance', 'Action', 'Thriller']
1651


'BPR: Top recommendations'

,Title,Drama,Comedy,Romance,Action,Thriller
ItemID,,,,,,
588,Beauty and the Beast (1991),0,0,0,0,0
257,Men in Black (1997),0,1,0,1,0
237,Jerry Maguire (1996),1,0,1,0,0
255,My Best Friend's Wedding (1997),0,1,1,0,0
95,Aladdin (1992),0,1,0,0,0
283,Emma (1996),1,0,1,0,0
70,Four Weddings and a Funeral (1994),0,1,1,0,0
25,"Birdcage, The (1996)",0,1,0,0,0
121,Independence Day (ID4) (1996),0,0,0,1,0


'WMF: Top recommendations'

,Title,Drama,Comedy,Romance,Action,Thriller
ItemID,,,,,,
313,Titanic (1997),1,0,1,1,0
272,Good Will Hunting (1997),1,0,0,0,0
258,Contact (1997),1,0,0,0,0
237,Jerry Maguire (1996),1,0,1,0,0
318,Schindler's List (1993),1,0,0,0,0
66,While You Were Sleeping (1995),0,1,1,0,0
50,Star Wars (1977),0,0,1,1,0
300,Air Force One (1997),0,0,0,1,1
402,Ghost (1990),0,1,1,0,1


Now that we have seen the top recommendations of the BPR and WMF models, let's do a comparison by taking a look at the genre distribution.

##### Comparing Models by Genre Distribution

In [19]:
# Let's introduce `combined_df` for comparison.
# This dataframe will be used to compare models by summing up genres from recommendations of different models
combined_df = pd.DataFrame({
    "Train Data %": filter_df["%"],
    "BPR Sum": bpr_df.select_dtypes(np.number).sum(), # group by genres, then get sum of each genre
    "WMF Sum": wmf_df.select_dtypes(np.number).sum() # likewise for WMF
})

# Get percentages of movie genre sums
combined_df['BPR %'] = combined_df['BPR Sum'] / combined_df['BPR Sum'].sum() * 100 
combined_df["WMF %"] = combined_df["WMF Sum"] / combined_df["WMF Sum"].sum() * 100

combined_df = combined_df.round(1) # round all 
combined_df = combined_df.sort_values("Train Data %", ascending=False)

# Let's take a look at the genre distribution by percentages
display("Train Data to Recommended % Distribution", combined_df[['Train Data %', 'BPR %', 'WMF %']])

'Train Data to Recommended % Distribution'

,Train Data %,BPR %,WMF %
Drama,24.9,18.8,20.7
Comedy,13.7,18.8,18.9
Romance,11.2,16.2,18.9
Action,10.5,10.3,5.4
Thriller,10.2,3.4,5.4
Adventure,6.7,7.7,2.7
War,5.3,6.0,7.2
Crime,4.2,0.9,0.9
Sci-Fi,3.2,6.0,5.4
Mystery,2.8,0.9,1.8


Now that we have seen the distribution of individual models, we are curious about what kind of distribution we will get from ensembling these models.

Let's see what happens when we ensemble these two models. 

## 2. Simple model ensembling by Borda Count

We will ensemble the two models using the Borda Count method. The Borda Count method is a simple voting method that ranks the items based on the sum of their ranks from each model.

Assuming that we have a list of 5 items, the Borda Count method works as follows:

1. For each model, rank the items from 1 to 5 based on the predicted scores.
2. Sum the ranks of each item across all models.
3. Sort the items based on the sum of their ranks.
4. The top-ranked item is the final recommendation.
5. Repeat the process for the next user.

Given the below example for a random user 123:

| Rank | Model 1 | Model 2 | Model 3 | Allocated Points (N - rank) |
|------|---------|---------|---------|-----------------------------|
| 1    | A       | D       | E       | 5 - 1 = 4                   |
| 2    | B       | C       | A       | 5 - 2 = 3                   |
| 3    | C       | A       | B       | 5 - 3 = 2                   |
| 4    | D       | B       | D       | 5 - 4 = 1                   |
| 5    | E       | E       | C       | 5 - 5 = 0                   |

Based on the allocated points for each of the items, we sum the points up to get our Borda Count.

| Item | Model 1 | Model 2 | Model 3 | Borda Count  |
|------|---------|---------|---------|--------------|
| A    | 4       | 2       | 3       | 9            |
| B    | 3       | 1       | 2       | 6            |
| C    | 2       | 3       | 0       | 5            |
| D    | 1       | 4       | 1       | 6            |
| E    | 0       | 0       | 4       | 4            |

New ranking: A > B, D > C > E


Lets implement this method below.

In [24]:
# Let's create a new dataframe to calculate ranking and borda count
rank_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

total_items = len(rank_df) # 1651 items

# Obtain points (inverse of rank) of the items based on the BPR score
rank_df["BPR Score"] = bpr_scores
rank_df["BPR Rank"] = rank_df["BPR Score"].rank(ascending=False).astype(int) # Get Rank where 1 = Top recommendation
rank_df["BPR Points"] = total_items - rank_df["BPR Rank"] # Get points by calculating ('Total Item count' - 'Rank')

# Do likewise for WMF
rank_df["WMF Score"] = wmf_scores
rank_df["WMF Rank"] = rank_df["WMF Score"].rank(ascending=False).astype(int) # Get Rank where 1 = Top recommendation
rank_df["WMF Points"] = total_items - rank_df["WMF Rank"] # Get points by calculating ('Total Item count' - 'Rank')

# Get Borda Count by summing up points of BPR and WMF
rank_df["Borda Count"] = rank_df["BPR Points"] + rank_df["WMF Points"]

# Round decimal places for readability purposes
rank_df = rank_df.round(3)

# Now let's take a look at the table with Borda Count 
display(rank_df.head(5))

,ItemID,BPR Score,BPR Rank,BPR Points,WMF Score,WMF Rank,WMF Points,Borda Count
0,381,2.050,273,1378,4.228,125,1526,2904
1,602,0.132,739,912,2.589,539,1112,2024
2,431,-0.244,829,822,3.169,383,1268,2090
3,875,1.237,433,1218,1.918,748,903,2121
4,182,1.505,378,1273,3.581,265,1386,2659


Now that we have Borda Count, let's rerank this list and to provide the ensembled model's recommendation.

In [25]:
# Introduce reranked dataframe for borda count
reranked_df = rank_df.sort_values("Borda Count", ascending=False)

# Let's take a look at the ensembled top 5 recommendations and their respective ranks
display("Re-ranked Top K Item recommendations", reranked_df.head(5))

'Re-ranked Top K Item recommendations'

,ItemID,BPR Score,BPR Rank,BPR Points,WMF Score,WMF Rank,WMF Points,Borda Count
425,237,5.187,3,1648,5.190,4,1647,3295
37,318,4.687,10,1641,5.150,5,1646,3287
305,181,4.608,12,1639,5.012,13,1638,3277
325,588,5.449,1,1650,4.805,31,1620,3270
92,50,4.148,25,1626,5.092,7,1644,3270


The top recommendation ItemID **181**, was ranked **7** on BPR and **12** on WMF.

By doing Borda Count, we are able to aggregate model recommendations.

Next, let's add the recommendations into the genre distribution dataframe to compare its results to the base models.

In [26]:
borda_count_topk = reranked_df["ItemID"].values[:TOPK] # Get top K (50) Item IDs

borda_df = item_df.loc[[int(i) for i in borda_count_topk]] # Filter genre data frame by the top item IDs

# Add Borda Count results into 'combined_df' dataframe for comparison
combined_df["Borda Count Sum"] = borda_df.select_dtypes(np.number).sum() # group by genre, and calculate sum of each genre
combined_df["Borda Count %"] = combined_df["Borda Count Sum"] / combined_df["Borda Count Sum"].sum() * 100 # Calculate percentage of sum to total
combined_df["Borda Count %"] = combined_df["Borda Count %"].round(1) # rounding for readability purposes

# Let's take a look at the genre distribution of train data, BPR, WMF and the newly added Borda Count
display("Borda Count Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %"]])

'Borda Count Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %
Drama,24.9,18.8,20.7,16.1
Comedy,13.7,18.8,18.9,19.5
Romance,11.2,16.2,18.9,14.4
Action,10.5,10.3,5.4,10.2
Thriller,10.2,3.4,5.4,5.1
Adventure,6.7,7.7,2.7,6.8
War,5.3,6.0,7.2,5.9
Crime,4.2,0.9,0.9,0.8
Sci-Fi,3.2,6.0,5.4,5.9
Mystery,2.8,0.9,1.8,0.8


In the next section, we will further add more models to the ensemble.

## 3. Adding more models to the Borda Count ensemble

We can easily add more models to the ensemble by training them and adding them. One approach is to train a model with different initializations using different random seeds. By adding multiple similar models of different random seeds (`seed=123`), some models could perform better for a set of users, while other models could perform better for another set of users.

Another way is to change the number of latent factors `k`. By achieving better performance on a subset of data for each of the models, we are able to improve the performance of the ensembled model as a whole.

By ensembling these models, we could potentially achieve better performance when combined.

Let's try adding a few more similar models with different random seed initializations.

In [21]:
# WMF models with different seeds
wmf_model_123 = WMF(name="WMF_123", k=10, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=123)
wmf_model_456 = WMF(name="WMF_456", k=20, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=456)
wmf_model_789 = WMF(name="WMF_789", k=30, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=789)
wmf_model_888 = WMF(name="WMF_888", k=40, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=888)
wmf_model_999 = WMF(name="WMF_999", k=50, max_iter=300, a=1.0, b=0.1, learning_rate=0.001, lambda_u=0.01, lambda_v=0.01, seed=999)

models = [wmf_model_123, wmf_model_456, wmf_model_789, wmf_model_888, wmf_model_999]

# Let's run an experiment to take a look at how different these models are, with just different random seeds!
experiment = Experiment(rs, models, metrics, user_based=True).run()


[WMF_123] Training started!


100%|██████████| 300/300 [00:16<00:00, 18.31it/s, loss=173]


Learning completed!

[WMF_123] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 1773.87it/s]



[WMF_456] Training started!


...


[WMF_999] Training started!


100%|██████████| 300/300 [00:24<00:00, 12.47it/s, loss=102] 


Learning completed!

[WMF_999] Evaluation started!


Ranking: 100%|██████████| 940/940 [00:00<00:00, 2131.94it/s]


TEST:
...
        | Precision@50 | Recall@50 | Train (s) | Test (s)
------- + ------------ + --------- + --------- + --------
WMF_123 |       0.1750 |    0.5102 |   16.6207 |   0.5441
WMF_456 |       0.1779 |    0.5219 |   16.9955 |   0.4945
WMF_789 |       0.1736 |    0.5128 |   17.2604 |   0.5317
WMF_888 |       0.1707 |    0.5031 |   17.5373 |   0.4779
WMF_999 |       0.1656 |    0.4937 |   24.2003 |   0.4467



Based on the results, we can see that even within the same model, the results can vary. 

Let's try ensembling all these models together into 1 single model by Borda Count, and look at its recommendations.

In [22]:
# Let's create a different dataframe to calculate ranking and borda count
rank_2_df = pd.DataFrame({
    "ItemID": item_idx2id,
})

# Add a column named 'Ensembled WMF Model'
rank_2_df["Ensembled WMF Model"] = 0

# Calculate the points (inverse of rank) for each of the models and accumulate them into the 'Enhanced Borda Count' column
for model in models:
    name = model.name
    recommendations, scores = model.rank(UIDX)
    rank_2_df[name + "_rating"] = scores
    rank_2_df[name + "_rank"] = rank_2_df[name + "_rating"].rank(ascending=False).astype(int)
    rank_2_df[name + "_points"] = total_items - rank_2_df[name + "_rank"]
    rank_2_df["Ensembled WMF Model"] = rank_2_df["Ensembled WMF Model"] + rank_2_df[name + "_points"]

# Round results for readability
rank_2_df = rank_2_df.round(3)

# Let's sort and view the top recommendations!
print("Model score calculation:")
display(rank_2_df[["Ensembled WMF Model"]].sort_values("Ensembled WMF Model", ascending=False).head(10))

Model score calculation:


,Ensembled WMF Model
37,8226
197,8179
152,8150
279,8125
156,8122
532,8115
147,8109
522,8081
61,8070
670,8066


In [27]:
# Now, let's add them to the combined dataframe for comparison with earlier models
enhanced_borda_count_topk = list(rank_2_df.sort_values("Ensembled WMF Model", ascending=False)["ItemID"].values[:TOPK])
enhanced_borda_df = item_df.loc[[int(i) for i in enhanced_borda_count_topk]]

combined_df["Ensembled WMF Model Sum"] = enhanced_borda_df.select_dtypes(np.number).sum()
combined_df["Ensembled WMF Model %"] = combined_df["Ensembled WMF Model Sum"] / combined_df["Ensembled WMF Model Sum"].sum() * 100
combined_df["Ensembled WMF Model %"] = combined_df["Ensembled WMF Model %"].round(1)

# Let's compare the recommendation distribution
display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Ensembled WMF Model %"]])

'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Ensembled WMF Model %
Drama,24.9,18.8,20.7,16.1,24.6
Comedy,13.7,18.8,18.9,19.5,14.0
Romance,11.2,16.2,18.9,14.4,15.8
Action,10.5,10.3,5.4,10.2,9.6
Thriller,10.2,3.4,5.4,5.1,7.0
Adventure,6.7,7.7,2.7,6.8,7.0
War,5.3,6.0,7.2,5.9,5.3
Crime,4.2,0.9,0.9,0.8,5.3
Sci-Fi,3.2,6.0,5.4,5.9,5.3
Mystery,2.8,0.9,1.8,0.8,3.5


Now that we have touched on simple borda count, let's see how we could use other methods and popular packages such as **scikit-learn** to do advanced model ensembling.

## 4. Model Ensembling via Regression Models

We could continue by thinking of this as a meta-learning problem. We could treat recommendations of each base model as features and train a meta-learner to predict the final recommendation.

This could be any ML model such as a Linear Regression, Random Forest, Gradient Boosting, or even a Neural Network.

In this example, we will use a simple Linear Regression model to predict the final recommendation.

We will teach a model to learn from the different outputs of the base models of WMF.

##### 4.1 Prepare Data

In [28]:
# First, lets create training and test data dataframes
training_df = pd.DataFrame(zip(*train_set.uir_tuple)) # Add 'User Index', 'Item Index', 'Rating' triples as records in dataframe
training_df.columns = ['user_idx', 'item_idx', 'ground_rating'] # Set column names

# Get all possible user_index, item_index combinations, add them into dataframe for inference
all_df = pd.DataFrame({
    "user_idx": [user_idx for user_idx in range(train_set.num_users) for _ in range(train_set.num_items)],
    "item_idx": [item_idx for _ in range(train_set.num_users) for item_idx in range(train_set.num_items)],
})
all_df['item_id'] = all_df.apply(lambda row: item_idx2id[int(row['item_idx'])], axis=1) # Add 'Item ID' column into dataframe by converting 'Item Index' to 'Item ID'

# Lets get all the scores for the models trained in Part 3.

# For each model, we add individual predicted ratings by individual models to training and test dataframes
for model in models:
    name = model.name
    training_df[name + "_rating"] = training_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1) # for training
    all_df[name + "_rating"] = all_df.apply(lambda row: model.score(int(row['user_idx']), int(row['item_idx'])), axis=1) # for inference

# Let's pick out the 5 features - predicted ratings from the 5 models trained
X_train = training_df[['WMF_123_rating', 'WMF_456_rating', 'WMF_789_rating', 'WMF_888_rating', 'WMF_999_rating']]
y_train = training_df['ground_rating'] # use ground truth to train this linear regression model
X_inference = all_df[['WMF_123_rating', 'WMF_456_rating', 'WMF_789_rating', 'WMF_888_rating', 'WMF_999_rating']] # all data, used to predict values for ranking

display("Training features", X_train.head(3)) # predicting ratings as features
display("Target values", y_train.head(3)) # ground truth ratings
display("Inference Data", X_inference.head(3)) # all inference data

'Training features'

,WMF_123_rating,WMF_456_rating,WMF_789_rating,WMF_888_rating,WMF_999_rating
0,2.109998,3.753301,3.692581,3.769731,3.974400
1,2.791594,2.339869,2.418739,2.745775,3.102975
2,3.750995,3.536710,3.439668,3.409918,3.865470


'Target values'

0    4.0
1    3.0
2    4.0
Name: ground_rating, dtype: float64

'Inference Data'

,WMF_123_rating,WMF_456_rating,WMF_789_rating,WMF_888_rating,WMF_999_rating
0,2.109998,3.753301,3.692581,3.769731,3.974400
1,0.807329,-0.628916,-0.062733,-0.281984,-0.589187
2,1.648487,1.839234,1.828926,1.867302,1.548658


Now that we have already prepared the data for fitting into a **scikit-learn** model, let's first try to train a Linear Regression model

##### 4.2 Fitting Linear Regression Model

In [34]:
# Let's now fit into a Linear Regression model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train) # train the model

# Input: 5 base model predicted ratings. Output: final predicted rating based on linear regression
y_pred = regr.predict(X_inference) # Get predictions based on trained model

all_df["Linear Regression Rating"] = y_pred # create a column in `test_df` for the predictions

# Get Top K ratings from predictions
all_df = all_df.sort_values("Linear Regression Rating", ascending=False) # sort by predicted ratings
top_item_ids = all_df[all_df['user_idx'] == UIDX]['item_id'].values[:TOPK] # filter top K (50 as set in Section 2.3)

# Place them into the comparison distribution dataframe
linear_regression_df = item_df.loc[[int(i) for i in top_item_ids]] # Get genres of ratings
combined_df["Linear Regression Sum"] = linear_regression_df.select_dtypes(np.number).sum() # group by genre and sum them up
combined_df["Linear Regression %"] = combined_df["Linear Regression Sum"] / combined_df["Linear Regression Sum"].sum() * 100 # get percentages of (genre sum / whole sum)

combined_df["Linear Regression %"] = combined_df["Linear Regression %"].round(1) # round values for readability

# Now let's take a look at how the genre distribution is
display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Ensembled WMF Model %", "Linear Regression %"]])

'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Ensembled WMF Model %,Linear Regression %
Drama,24.9,18.8,20.7,16.1,24.6,27.9
Comedy,13.7,18.8,18.9,19.5,14.0,7.7
Romance,11.2,16.2,18.9,14.4,15.8,13.5
Action,10.5,10.3,5.4,10.2,9.6,6.7
Thriller,10.2,3.4,5.4,5.1,7.0,10.6
Adventure,6.7,7.7,2.7,6.8,7.0,7.7
War,5.3,6.0,7.2,5.9,5.3,4.8
Crime,4.2,0.9,0.9,0.8,5.3,4.8
Sci-Fi,3.2,6.0,5.4,5.9,5.3,5.8
Mystery,2.8,0.9,1.8,0.8,3.5,1.9


We have successfully fit into a Linear Regression model using the 5 WMF base models built on Cornac.

You could continue using different regression models on **scikit-learn** to further predict ratings.

In the section below, we will train a Random Forest model.

##### 4.3 Fitting Random Forest Model



In [35]:
# Let's now train a Random Forest model
randomforest_model = RandomForestRegressor(n_estimators=500, random_state=42) 
randomforest_model.fit(X_train, y_train) # Train the model

# Input: 5 base model predicted ratings. Output: final predicted rating based on random forest
y_pred = randomforest_model.predict(X_inference)

all_df["Random Forest Rating"] = y_pred # create a column in `test_df` for the predictions

# Get Top K ratings from predictions
all_df = all_df.sort_values("Random Forest Rating", ascending=False) # sort by predicted ratings
top_item_ids = all_df[all_df['user_idx'] == UIDX]['item_id'].values[:TOPK] # filter top K (50 as set in Section 2.3)

# Place them into the comparison distribution dataframe
random_forest_df = item_df.loc[[int(i) for i in top_item_ids]] # Get genres of ratings
combined_df["Random Forest Sum"] = random_forest_df.select_dtypes(np.number).sum() # group by genre and sum them up
combined_df["Random Forest %"] = combined_df["Random Forest Sum"] / combined_df["Random Forest Sum"].sum() * 100 # get percentages of (genre sum / whole sum)

combined_df["Random Forest %"] = combined_df["Random Forest %"].round(1) # round values for readability

# Now let's take a look at how the genre distribution is
display("Combined Recommendations Distribution", combined_df[["Train Data %", "BPR %", "WMF %", "Borda Count %", "Ensembled WMF Model %", "Linear Regression %", "Random Forest %"]])

'Combined Recommendations Distribution'

,Train Data %,BPR %,WMF %,Borda Count %,Ensembled WMF Model %,Linear Regression %,Random Forest %
Drama,24.9,18.8,20.7,16.1,24.6,27.9,27.8
Comedy,13.7,18.8,18.9,19.5,14.0,7.7,14.4
Romance,11.2,16.2,18.9,14.4,15.8,13.5,10.0
Action,10.5,10.3,5.4,10.2,9.6,6.7,8.9
Thriller,10.2,3.4,5.4,5.1,7.0,10.6,7.8
Adventure,6.7,7.7,2.7,6.8,7.0,7.7,4.4
War,5.3,6.0,7.2,5.9,5.3,4.8,4.4
Crime,4.2,0.9,0.9,0.8,5.3,4.8,4.4
Sci-Fi,3.2,6.0,5.4,5.9,5.3,5.8,3.3
Mystery,2.8,0.9,1.8,0.8,3.5,1.9,1.1


What happened in Section 4 was that we trained a linear regression model and random forest model to learn from the different base models, adapting to the changes in features based on how these base models are configured to be.

---

Ensemble learning doesn't stop here. You could continue on by: 
- Trying different Cornac base models
- Trying different ensemble methods
- Tweaking around base models and different parameters within them

By having different base models with different specializations, model ensembling could leverage on the strengths of different models. Effectiveness of these models could depend on many factors such as diversity and quality of base models, dataset size and quality.

It is also important to note that while model ensembling theoretically provides superior performance, there could be instances where base models outperforms ensembled models. Model ensembling also requires more computational resources as well. Therefore, we should consider striking a balance between performanace and computational costs.

---

So what constitutes to a good ensembled model? Which base models and configurations are ideal? These are topics that require further experimentation and discussion.

## 5. Experimental Comparison